# Step on a Platinum (211) surface

## 0. Introduction

This notebook demonstrates how to create a slab of Pt(211) which will show the step from the manuscript:

> **Šljivančanin, Ž., & Hammer, B.**
> "Oxygen dissociation at close-packed Pt terraces, Pt steps, and Ag-covered Pt steps studied with density functional theory."
> Surface Science, 515(1), 235–244. 
> [DOI: 10.1016/s0039-6028(02)01908-8 ](https://doi:10.1016/s0039-6028(02)01908-8 )

Replicating the material from the FIG. 1. B.

<img src="https://github.com/Exabyte-io/documentation/raw/12617167278ae3523adc028583b21ea4e8ebd197/images/tutorials/materials/defects/defect_surface_step_platinum/0-figure-from-manuscript.webp" alt="Pt(211) step" width="400"/>




## 1. Prepare the Environment
### 1.1. Set up defect parameters 

In [ ]:
MATERIAL_NAME = "Pt"
MILLER_INDICES = (2, 1, 1)
THICKNESS = 6  # in atomic layers
VACUUM = 10.0  # in angstroms
XY_SUPERCELL_MATRIX = [[1, 0], [0, 1]]
USE_ORTHOGONAL_C = True
USE_CONVENTIONAL_CELL = True

VISUALIZATION_REPETITIONS = [1, 1, 1]

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install("mat3ra-api-examples @ git+https://github.com/Exabyte-io/api-examples.git@epic/SOF-7623", deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("specific_examples")

### 1.3. Get input materials

In [ ]:
from mat3ra.made.tools.build import MaterialWithBuildMetadata
from mat3ra.standata.materials import Materials

material = MaterialWithBuildMetadata.create(Materials.get_by_name_first_match(MATERIAL_NAME))

### 1.4. Preview the material

In [ ]:
from utils.visualize import visualize_materials as visualize

visualize(material, repetitions=VISUALIZATION_REPETITIONS, rotation="0x")
visualize(material, repetitions=VISUALIZATION_REPETITIONS, rotation="-90x")

## 2. Create a step with a slab

### 2.1. Create slab configuration
Slab Configuration lets define the slab thickness, vacuum, and the Miller indices of the interfacial plane and get the slabs with possible terminations.


In [ ]:
from mat3ra.made.tools.build.slab.helpers import create_slab

slab_211 = create_slab(
    crystal=material,
    miller_indices=MILLER_INDICES,
    number_of_layers=THICKNESS,
    vacuum=VACUUM,
    use_orthogonal_c=USE_ORTHOGONAL_C,
    use_conventional_cell=USE_CONVENTIONAL_CELL,
    xy_supercell_matrix=XY_SUPERCELL_MATRIX
)

### 2.3. Visualize the slab

In [ ]:
visualize(slab_211, repetitions=VISUALIZATION_REPETITIONS, rotation="0x")
visualize(slab_211, repetitions=VISUALIZATION_REPETITIONS, rotation="-90x")

## 3. Create a step with terrace builder
### 3.1. Setup terrace parameters

In [ ]:
from types import SimpleNamespace
# Material selection
# Which material to use from input list
MATERIAL_INDEX = 0

# Terrace parameters:
# Normal vector describing a plane that cuts the terrace from added layers (Miller indices)
CUT_DIRECTION = [0, 1, 1]
# Point the cutting plane passes through, in crystal coordinates
PIVOT_COORDINATE = [0.0, 0.0, 0.25]
# Height of terrace in atomic layers
NUMBER_OF_ADDED_LAYERS = 1
# Use cartesian instead of crystal coordinates
USE_CARTESIAN_COORDINATES = False
# Rotate to match periodic boundary conditions
ROTATE_TO_MATCH_PBC = True

# Slab parameters for creating a new slab if provided material is not a slab
SLAB_PARAMETERS = SimpleNamespace(
    miller_indices= (1, 1, 1),
    number_of_layers= 6,  # Number of layers in the slab
    thickness= 6,
    vacuum= 15.0,
    use_orthogonal_c= True,
    xy_supercell_matrix= [[2, 0], [0, 2]]
)

# Visualization parameters
SHOW_INTERMEDIATE_STEPS = True
# Structure repeat in view
CELL_REPETITIONS_FOR_VISUALIZATION = [1, 1, 1]

### 3.2. Configure a terrace defect

In [ ]:
from mat3ra.made.tools.build.defect.terrace.helpers import create_terrace

slab_111 = create_slab(
    crystal=material,
    miller_indices=SLAB_PARAMETERS.miller_indices,
    number_of_layers=SLAB_PARAMETERS.number_of_layers,
    vacuum=SLAB_PARAMETERS.vacuum,
    use_orthogonal_c=SLAB_PARAMETERS.use_orthogonal_c,
    xy_supercell_matrix=SLAB_PARAMETERS.xy_supercell_matrix
)

terrace_slab = create_terrace(
    slab=slab_111,
    cut_direction=CUT_DIRECTION,
    pivot_coordinate=PIVOT_COORDINATE,
    number_of_added_layers=NUMBER_OF_ADDED_LAYERS,
    use_cartesian_coordinates=USE_CARTESIAN_COORDINATES,
    rotate_to_match_pbc=ROTATE_TO_MATCH_PBC
)


### 3.4. Visualize the terrace defect

In [ ]:
visualize(terrace_slab, viewer="wave")

## 4. Download materials

In [ ]:
from utils.jupyterlite import download_content_to_file, set_materials

set_materials([slab_211, terrace_slab])
download_content_to_file(slab_211, "Pt_211_step.json")
download_content_to_file(terrace_slab, "Pt_111_terrace.json")